# Coverage Report Generation V9

In [ ]:
#import imp
#imp.reload(utilsc)

## 1. Variables to modify

In [ ]:
import json

auth_path = r".\credentials.json"
auth_path = auth_path.replace("\\","/")

with open(auth_path) as json_file:
    json_credentials = json.load(json_file)
    json_file.close()

user = json_credentials["cisco_tableau"]["user"]    # Your Cisco e-mail address
cr_tableau_template_name = json_credentials["paths"]["tableau_template_cover_path"] # twb template name/path
personal_access_token_name = json_credentials["cisco_tableau"]["access_token_name"]
personal_access_token_secret = json_credentials["cisco_tableau"]["token"]

## 2. Libraries & Functions

In [ ]:
%%time
# ----------------------------------------- LIBRARIES

#!pip install snowflake-connector-python
#!pip install tableauhyperapi
#!pip install tableau-api-lib
#!pip install tableauserverclient
#######!pip install hyperapi
#!pip install fastparquet

import os
import datetime
import pandas as pd
import numpy as np
import itertools
import shutil
import xml.etree.ElementTree as ET
import snowflake.connector
import re
import webbrowser

from zipfile import ZipFile
import tableauserverclient as TSC
from tableauhyperapi import *

import smartsheet_lib as smartsheet
import utils_coverage as utilsc

# ----------------------------------------- FUNCTIONS

qs = {8:1,9:1,10:1,11:2,12:2,1:2,2:3,3:3,4:3,5:4,6:4,7:4}
fy = {1:1,2:1,3:0,4:0}

class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

## 3. Generating the Workbooks

In [ ]:
%%time

current_path = os.getcwd().replace("\\","/")

date = datetime.datetime.today()
date = date.date()

if date.weekday() in [0,1]:
    delta_t = str(date-datetime.timedelta(4))
else:
    delta_t = str(date-datetime.timedelta(1))
    
month = datetime.datetime.today().strftime("%B")

print ('Current Path: ' + str(current_path))
print ('Current Date: ' + str(date))

# -----------------------------------------------------------------

smartsheet_client = smartsheet.init_conn("J937aTx1hHKkv3ju15G0XGC0Qz6af50MNrhLq")
oa_sheet = smartsheet.load_sheet(3815245262153604,client=smartsheet_client,modified_since=delta_t)
oa_df = pd.DataFrame()
sheet_new = smartsheet.get_last_n_rows(oa_sheet,n_rows=4000)
oa_df = smartsheet.sheet_to_df2(sheet_new,columns=oa_sheet.columns)
oa_df = oa_df.query("`Request ID` != ''")
#oa_df['Request ID'] = oa_df['Request ID'].apply(lambda x:int(x)) # drop request id decimal places

fields_df = utilsc.get_da_requests(da=user,df=oa_df)
fields_df['CR Creation Date'] = date
fields_df

In [ ]:
%%time
# Retrieving snowflake data and creating dataframes
savs_list,gu_list,cav_list,cr_list = utilsc.get_ids_list(fields_df,separator=',') # IDs to look for on Snowflake
print('1/7 - IDs created ...')
uncovered_data = utilsc.get_uncovered_data2(user,ids_sav=savs_list,ids_gu=gu_list,ids_cr=cr_list,ids_cav=cav_list)  # Retrieve Uncovered Data from SF
#uncovered_data = uncovered_data[uncovered_data['LINE_STATUS'].isin(['MISS ATTACH 1mo - 3mo','MISS ATTACH 3mo - 12mo' ])]
print('2/7 - Uncovered data retrieved ...')
appliance_data = utilsc.get_appliance_data(user,ids_sav=savs_list,ids_gu=gu_list,ids_cr=cr_list,ids_cav=cav_list)  # Retrieve Appliance Data from SF
print('3/7 - Appliance data retrieved ...')
coverage_data = utilsc.get_coverage_data(user,ids_sav=savs_list,ids_gu=gu_list,ids_cr=cr_list,ids_cav=cav_list) # Retrieve Coverage Data from SF
print('4/7 - Coverage data retrieved ...')
contracts_data = utilsc.get_contracts_data(user,ids_sav=savs_list,ids_gu=gu_list,ids_cr=cr_list,ids_cav=cav_list) # Retrieve Contracts Data from SF
print('5/7 - Contracts data retrieved ...')
tac_data = utilsc.get_tac_data(user,ids_sav=savs_list,ids_gu=gu_list,ids_cr=cr_list,ids_cav=cav_list) # Retrieve TAC data
print('6/7 - TAC data retrieved ...')
df_uncovered,df_coverage,df_contracts,df_appliance = utilsc.format_columns(uncovered_data,coverage_data,contracts_data,appliance_data)
print('7/7 - Data gathering finished')

In [ ]:
%%time
sns_df = pd.DataFrame()
folders_list = []
smartsheet_fields = ['index','Request ID','Date Created',
                         'Assigned DA','Campaign Name','Customer Name',
                         'Input file URL','ID TYPE','SAV ID','CAV ID',
                         'CAV BU ID','GU ID','Lvl1','Lvl2 (Region)',
                         'Contract ID','Inventory Name','Appliance ID',
                         'CR Party Name','CR Party ID','Comments','DA Comments',
                         'Status','Requester Name','Who should be notified on completion of Analysis',
                         'OP Status','CR Creation Date']

#Empty list to store the Recommended Estimate for each account
Recommended_Estimate_List = []
Initial_Estimate_List = []

for idx in range(len(fields_df)):
    name = re.sub('[^A-Za-z0-9\-]+', '', fields_df['Customer Name'][idx][0:15])
    theater = fields_df['Lvl1'][idx]
    req_type = fields_df['ID TYPE'][idx]
    
    if req_type == 'SAV ID':
        ids = fields_df['sav_list'][idx]
        id_type = 'SAV'
    elif req_type == 'GU ID':
        ids = fields_df['gu_list'][idx]
        id_type = 'GU'
    elif req_type == 'CR Party ID':
        ids = fields_df['cr_list'][idx]
        id_type = 'CR'
    elif req_type == 'CAV ID':
        ids = fields_df['cav_list'][idx]
        id_type = 'CAV'   
    
    
    req_id = str(int(fields_df['Request ID'][idx]))
    date_created = fields_df['CR Creation Date'][idx]
    print(f"{idx+1}/{len(fields_df)}: {name}-{req_id}",end=" - ")
    
    folder =  f"CR_{theater}_{req_id}_{name}_{req_type}_{str(date.year)}_{str(date.month)}_{str(date.day)}" # folder name
    folder_path = f"{current_path}/CR/{month}/{date}/{folder}"    
    
    uncovered_data_filtered = df_uncovered[df_uncovered['CUSTOMER_ID'].isin([int(customer_id) for customer_id in ids])]
    uncovered_data_filtered = uncovered_data_filtered[uncovered_data_filtered['ACCOUNT_IDENTIFIER'] == id_type]
    
    appliance_data_filtered = df_appliance[df_appliance['CUSTOMER_ID'].isin([int(customer_id) for customer_id in ids])]
    appliance_data_filtered = appliance_data_filtered[appliance_data_filtered['CUSTOMER_IDENTIFIER'] == id_type]   

    #Function to Calculate the Recommended Estimate
    Recommended_Estimate = utilsc.Total_Recommended_Estimate_CR(uncovered_data_filtered)
    Recommended_Estimate_SSPT = utilsc.Total_Recommended_Estimate_CR(uncovered_data_filtered)
    Initial_Estimate = utilsc.Total_Initial_Estimate(uncovered_data_filtered)
    
    
    if len(uncovered_data_filtered) == 0:
        print(bcolors.FAIL + f"No Uncovered Data - No Serial Numbers to upload" + bcolors.ENDC)
        folders_list.append("N/A")
            
    else:
        
        if Recommended_Estimate == 0 and Recommended_Estimate_SSPT == 0:
            print(bcolors.FAIL + f"No Uncovered Oppty - No Serial Numbers to upload" + bcolors.ENDC)
            folders_list.append("N/A")
        else:
            
            try:
                os.makedirs(f"{folder_path}/Data")
            except: pass    
            folders_list.append(folder)

            coverage_data_filtered = df_coverage[df_coverage['CUSTOMER_ID'].isin([int(customer_id) for customer_id in ids])]
            coverage_data_filtered = coverage_data_filtered[coverage_data_filtered['ACCOUNT_IDENTIFIER'] == id_type]

            contracts_data_filtered = df_contracts[df_contracts['Bk Sales Account Id Int'].isin([int(customer_id) for customer_id in ids])]
            contracts_data_filtered = contracts_data_filtered[contracts_data_filtered['ACCOUNT_IDENTIFIER'] == id_type]

            smartsheet_filtered = fields_df[smartsheet_fields].iloc[[idx]]

            if id_type == 'CR':
                tac_filtered = tac_data[tac_data['FLAG']=='GU']
                tac_filtered = tac_data[tac_data['PARTY ID'].isin([int(customer_id) for customer_id in ids])]
            else:
                tac_filtered = tac_data[tac_data['FLAG']==id_type]
                tac_filtered = tac_filtered[tac_filtered['ID'].isin([int(customer_id) for customer_id in ids])]

            # Removing some columns
            contracts_data_filtered.drop(['ACCOUNT_IDENTIFIER'],axis = 1,inplace = True)   

            # Creating the extracts
            uncovered_data_filtered.set_index('ACCOUNT_ID').to_csv(f"{folder_path}/Data/uncovered.csv")
            appliance_data_filtered.set_index('APPLIANCE_ID').to_csv(f"{folder_path}/Data/appliance_details.csv")
            coverage_data_filtered.set_index('CUSTOMER_ID').to_csv(f"{folder_path}/Data/contracts-and-coverage.csv")
            contracts_data_filtered.set_index('Bk Sales Account Id Int').to_csv(f"{folder_path}/Data/contract-view.csv")
            smartsheet_filtered.to_csv(f"{folder_path}/Data/Smartsheet.csv")
            utilsc.create_extract(name='TAC',columns=utilsc.get_schema(table='TAC'),df=tac_filtered,path=folder_path)
            
            # -------------------------------------------------- Creating the workbooks twb
            tree = ET.parse(cr_tableau_template_name)
            for extract_path in tree.getroot().findall("./datasources/datasource/connection/named-connections/named-connection/connection[@class='textscan']"):
                extract_name = extract_path.attrib['directory']#.split('/')[-1]
                extract_path.attrib['directory'] = f"{folder_path}/Data"

            for extract_path in tree.getroot().findall("./datasources/datasource/connection/named-connections/named-connection/connection[@class='hyper']"):
                extract_name = extract_path.attrib['dbname'].split('/')[-1]
                extract_path.attrib['dbname'] = f"{folder_path}/Data/{extract_name}"

            ex_list = ["6 - TAC.hyper"]

            for new_extract_path in tree.getroot().findall("./datasources/datasource/extract/connection[@class='hyper']"):
                new_extract_name = new_extract_path.attrib['dbname'].split('/')[-1]
                if new_extract_name in ex_list:
                    new_extract_path.attrib['dbname'] = f"{folder_path}/Data/{new_extract_name}"


            try:
                with open (f"{folder_path}/{folder}.twb", "wb") as files :                                             
                    tree.write(files)

                #utilsc.convert_to_twbx(f"{folder_path}")
                print("twb created successfully")

                sn_df = uncovered_data_filtered[['SERIAL_NUMBER','CUSTOMER_ID','CUSTOMER_NAME','INSTANCE_NUMBER','LINE_STATUS','BK_PRODUCT_ID']]
                sn_df['USER_ID'] = user
                sn_df['SALES_LEVEL_1_NAME'] = theater
                sn_df['SALES_LEVEL_2_NAME'] = fields_df['Lvl2 (Region)'][idx]
                sn_df['COMPASS_REQ_ID'] = req_id
                sn_df['COMPLETION_QUARTER'] = f"FY{date_created.year + fy.get(qs.get(date_created.month))}Q{qs.get(date_created.month)}"
                sn_df['COMPLETION_DATE'] = date_created
                sn_df["COMPLETION_DATE"] = sn_df["COMPLETION_DATE"].astype(str)
                sn_df['ACCOUNT_IDENTIFIER'] = id_type
                sn_df = sn_df[['USER_ID','SERIAL_NUMBER','CUSTOMER_ID','CUSTOMER_NAME','ACCOUNT_IDENTIFIER','SALES_LEVEL_1_NAME','SALES_LEVEL_2_NAME','COMPASS_REQ_ID','COMPLETION_QUARTER','COMPLETION_DATE','INSTANCE_NUMBER','LINE_STATUS','BK_PRODUCT_ID']]
                print(f" - {len(sn_df)} Serial Numbers to append")
                sns_df = pd.concat([sns_df,sn_df])                
            except: print(bcolors.FAIL + "Error" + bcolors.ENDC)
    
    Recommended_Estimate_List.append(Recommended_Estimate)
    Initial_Estimate_List.append(Initial_Estimate)

# Uploading URLs

In [ ]:
folders_id = {
    "US COMMERCIAL": "fd7772ee-058d-463a-bd0a-4f65542801a4", # Americas
    "GLOBAL ENTERPRISE SEGMENT": "e57405f3-ca6c-4bc2-ad00-6e9e4eaddca3",
    "LATIN AMERICA": "11411241-08ec-488e-b5e5-b858051464a5",
    "CANADA": "2c7d703c-ccd7-4ebb-9a80-493efec42b37",
    "AMERICAS_SP": "001fdd45-0e78-4c94-8529-f541a58903b0",
    "US PS Market Segment": "bc2b9643-6dab-4659-967c-4c7faa1c9fbd",
    "ANZ AREA": "dc644422-3a51-4940-9e0c-5b9eeba4e938", # APJC
    "ASEAN_AREA": "486500cf-bab3-42f2-8cf6-912e0db03056",
    "GREATER_CHINA": "48eaf322-742a-4b72-a750-fe87a18a3e46",
    "INDIA_AREA": "3eda13f8-ce22-40fc-89e1-cca9f69318ee",
    "JAPAN__": "18c5b29f-d451-4546-b924-0f9964d98c95",
    "ROK_AREA": "0f98151c-0030-4f4b-90fc-a785b444cea1",
    "APJ_SP": "517cddf4-032f-4dc0-9569-23ce801ebd33",
    "EMEAR_GERMANY": "d11b1067-045d-42cb-a4c8-67805e4523a4", # EMEAR
    "EMEAR_SP": "35a5ac5e-1654-421e-bb75-d8c95a9408e3",
    "EMEAR-SOUTH": "7e91e72f-18bf-4b3b-8b72-7b964140743b",
    "EMEAR-NORTH": "34b13798-5c47-4bcf-b3d1-917d802d511f",
    "EMEAR-UKI": "268fdf46-d3ee-4623-ace9-97bb32ccc328",
    "EMEAR_MEA": "95dcde09-a6fc-4fb1-b7a8-69e1dc4775e8",
    "EMEAR-CENTRAL": "038970b3-8ea2-4a9a-86e8-6bd470350656"
}

fields_df2 = fields_df.copy()
fields_df2['folder_id'] = fields_df2['Lvl2 (Region)'].apply(lambda x : folders_id.get(x,''))
#fields_df2['folder_id'] = '2fffed65-9c48-4ac3-9c96-21b9c2b8ca8f'
fields_df2['project_name'] = folders_list
fields_df2['project_url'] = fields_df2['project_name'].apply(lambda x: utilsc.get_url(x))
fields_df2[['Customer Name','Lvl2 (Region)','folder_id','project_name','project_url']]

In [ ]:
%%time
# ---------------------------------------------------------

import os
import datetime

from tableau_api_lib import TableauServerConnection
from tableau_api_lib.utils import querying
import time

current_path = os.getcwd().replace("\\","/")

#date = datetime.datetime.today()
#date = date.date()

month = datetime.datetime.today().strftime("%B")

config = {
    "tableau_server": {
        'server': 'https://cx-tableau-stage.cisco.com',
        'api_version': '3.13',
        'personal_access_token_name': personal_access_token_name,
        'personal_access_token_secret': personal_access_token_secret,
        'site_name': 'Compass',
        'site_url': 'Compass'
    }
}

conn = TableauServerConnection(config,env='tableau_server')

for idx in range(len(fields_df2)):
    conn.sign_in()
    project = fields_df2['project_name'][idx]
    
    if project != "N/A":
        name = fields_df2['Customer Name'][idx]

    # -------------------------------------------------------------------    

        try: 

            response = conn.publish_workbook(
                project_id=fields_df2['folder_id'][idx],
                workbook_file_path=f"{current_path}/CR/{month}/{date}/{project}/{project}.twbx",
                #workbook_file_path=f"{current_path}/CR/{month}/{date}/{project}.twbx",
                workbook_name="{}".format(project),
                #connection_username = user,
                #connection_password =  
                #embed_credentials_flag = True
                #workbook_views_to_hide=['QA','Package Info'],
                #hide_view_flag=True
            )

            if (response.status_code != 201):
                print(bcolors.FAIL + bcolors.BOLD + f"{idx+1}/{len(fields_df2)}:The {name} Coverage Report has an error and has not been published - {response.text}"+bcolors.ENDC)
                raise AssertionError()

            else : print(f"{idx+1}/{len(fields_df2)}:The {name} The Coverage Report has been published")

        except Exception as e: 
            print(bcolors.FAIL + bcolors.BOLD + str(e)+bcolors.ENDC)
            pass

        time.sleep(3)
        conn.sign_out()

In [ ]:
#Print and Open CRs Links
print(bcolors.UNDERLINE + "-----------------------------COV URL--------------------------------" + bcolors.ENDC)
for link in fields_df2['project_url']:
    if link != "N/A":
        webbrowser.open(link)
        print(link)
    else:
        print('N/A')

#Print Recommended Estimate for each account
print()
print(bcolors.UNDERLINE + "--------------------------COV OPPTY VALUE----------------------------" + bcolors.ENDC)
for Recommended_Estimate in Recommended_Estimate_List:
    print(Recommended_Estimate, end = '' '\n')
    

In [ ]:
#Print Initial Estimate for each account
print()
print(bcolors.UNDERLINE + "--------------------------INITIAL ESTIMATE VALUE----------------------------" + bcolors.ENDC)
for Initial_Estimate in Initial_Estimate_List:
    print(Initial_Estimate, end = '' '\n')

# RUN THIS CELL ONCE PER BATCH!

In [ ]:
%%time
# Uploading Serial Numbers to Snowflake
try:
    utilsc.upload_data_to_sf(sns_df,user)
    print(f"{len(sns_df)} Serial Numbers uploaded")
except: print(bcolors.FAIL + "Error" + bcolors.ENDC)